In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec

import os
import contextily as ctx
import geopandas as gpd

In [ ]:
tod = 'PM'

route_filename = 'route_shapes_carta_2019_08_15_{}.shp'
tods = ['AM', 'OP', 'MD']

gtfs = {t: gpd.read_file(os.path.join('..', '..', 'data', 'gtfs', route_filename.format(t))).to_crs(epsg=3857) for t in tods}

rtes = gpd.read_file(os.path.join('..', '..', 'data', 'networks', 'client_provided', 'transit', 'base_2014', 'transit_routes_base.shp')).to_crs(epsg=3857)

rte_ids = []
[rte_ids.extend(list(gtfs[t]['route_id'].unique())) for t in tods]
rte_ids.extend(list(rtes[rtes['HW_{}'.format(tod)] > 0]['ROUTE']))
rte_ids = set(rte_ids)

for rte_id in rte_ids:
    print('Route: {}'.format(rte_id))
    
    gtfs_rt = {}
    rtes_rt = {}
    
    for tod in tods:
        fig, ax = plt.subplots(1,1, figsize=(4.1,2.5), dpi=200, constrained_layout=True)
        
        gtfs_rt[tod] = gtfs[tod][gtfs[tod]['route_id'] == rte_id]
        rtes_rt[tod] = rtes[(rtes['ROUTE'] == rte_id) & (rtes['HW_{}'.format(tod)] > 0)]

        if not gtfs_rt[tod].empty:
            gtfs_rt[tod].plot(ax=ax, alpha=1, lw=5.5, edgecolor='w')
            gtfs_rt[tod].plot(ax=ax, alpha=1, lw=4.5, edgecolor='k')
    
        if not rtes_rt[tod].empty:
            rtes_rt[tod].plot(ax=ax, alpha=.4, lw=2, linestyle='-',edgecolor='w')
            rtes_rt[tod].plot(ax=ax, alpha=1, lw=1, edgecolor='r')
            
        if not gtfs_rt[tod].empty and not rtes_rt[tod].empty:
            ctx.add_basemap(ax, source='https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}')

        ax.xaxis.set_visible(False)
        ax.yaxis.set_visible(False)
    
        ax.set_title('Period: {}'.format(tod), fontsize=10)
        # ax.set_title('Route: {}\nPeriod: {}'.format(rte_id, tod), fontsize=10)
    
        leg_pos = 'best'
        font_size = 6
    
        if rte_id in ['1']:
            leg_pos = 'upper right'
        
        if rte_id in ['3', '7', '15', '21', '28']:
            leg_pos = 'lower right'
        
        if rte_id in ['4', '8', '19']:
            leg_pos = 'lower left'
        
        if rte_id in ['9']:
            leg_pos = 'center left'
    
        if rte_id in ['10A', '10C', '10G', '14', '16']:
            leg_pos = 'upper left'
    
    
        # ax.legend(loc=leg_pos, prop={'size': font_size})

        out_file = os.path.join('..', '..', 'Data', 'networks', 'client_provided', 'transit', 
                                'base_2014', 'comparisons', 'transit_route_{}_{}_comparison.jpg'.format(rte_id, tod))
    
    #plt.tight_layout(0,0,0,0)
        plt.savefig(out_file, dpi=200)
        plt.close(fig)